# Neural network for dose measurement

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
%matplotlib tk

import matplotlib.pyplot as plt
import NNPytorchLightning as NNPL

Here loading trained models to compare the training and validation losses per model, and look at performance on unseen data. These models are as follows:

model_cubes: trained just on cube counts, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

model_cubes_profiles: trained on cube counts and profiles, with a batch size of 200 and 200 samples per data set, learning rate of 1e-3, for 500 epochs (for time)

In [2]:
coeffs = '/home/nr1315/Documents/Project/effective_dose_coeffs.h5'
energy_bins = '/home/nr1315/Documents/Project/MachineLearning/energy_bins.npy'

model_cubes = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_1/',torch.rand((1,1,64)),coeffs,energy_bins)

model_cubes_profiles = NNPL.LoadModel('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_5/',torch.rand((1,1,76)),coeffs,energy_bins)

Also need to load the loss curves separately, due to the way they are from the logs.

In [3]:
model_cubes_tloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_new_data_version_1-tag-train_loss.csv')
model_cubes_vloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_new_data_version_1-tag-val_loss.csv')

model_cubes_profiles_tloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_profiles_new_data_version_5-tag-train_loss.csv')
model_cubes_profiles_vloss = pd.read_csv('/home/nr1315/Downloads/run-model_cubes_profiles_new_data_version_5-tag-val_loss.csv')

We also define the directories from which to load the testing data, for convenience:

In [4]:
testing_data_dir = '/home/nr1315/Documents/Project/MachineLearning/TestingData/'

AmBe_counts = 'SimCubeCounts_AmBe_5_0-0-0-0-1-0_1500.npy'
AmLi_counts = 'SimCubeCounts_AmLi_5_0-0-0-0-1-0_1500.npy'
Cf_counts = 'SimCubeCounts_Cf252_6_0-0-0-0-1-0_1500.npy'

AmBe_target = 'SimEnergyBins_AmBe.npy'
AmLi_target = 'SimEnergyBins_AmLi.npy'
Cf_target = 'SimEnergyBins_Cf252.npy'

For each model in turn, we will look at the loss curves and prediction on AmBe, AmLi, and Cf-252. 

### Cubes only model, 500 epochs, 200 samples per dataset



In [5]:
NNPL.PlotLosses([model_cubes_tloss,model_cubes_vloss],['Training loss','Validation loss'],'Model with only cube counts')

![Loss curves](Plots/model_cubes_loss.png)

Here the model seems to be training well, although it does not appear to finish training after 500 epochs. 

We now look at the model prediction quality on some unseen data, using the compare_pred_true function. This also returns the testing loss of the prediction.

In [6]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

loss_fn = nn.MSELoss()

l1 = NNPL.compare_pred_true(model_cubes,testing_data_dir+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes only model',ax[0],24,np.load(energy_bins),1,loss_fn)
l2 = NNPL.compare_pred_true(model_cubes,testing_data_dir+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes only model',ax[1],24,np.load(energy_bins),1,loss_fn)
l3 = NNPL.compare_pred_true(model_cubes,testing_data_dir+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes only model',ax[2],24,np.load(energy_bins),1,loss_fn)

![Predictions of test data](Plots/model_cubes_prediction.png)

In [7]:
print("AmBe loss: {}".format(l1))
print("AmLi loss: {}".format(l2))
print("Cf-252 loss: {}".format(l3))

AmBe loss: 0.0878705158829689
AmLi loss: 0.04326992481946945
Cf-252 loss: 0.017681293189525604


Whilst the model loss continues to decrease, the model still has poor performance on the unseen sources,  predicting negative values in at least half the bins in all three cases. It generally seems to predict the greatest count in the region around the average energy of each source, i.e. above, below and at 1 MeV for AmBe, AmLi and Cf respectively, as previous iterations of the model did. One possible way to combat this may be to introduce more complex sources into the training set, such as linear combinations of existing monoenergetics, to help the network learn how to better reconstruct a more complicated fluence.

It is then informative to see the performance of the model on some of the validation data explicitly, as is shown below. The function used to plot this can be used to scroll through all of the training datasets.

In [8]:
from NNPytorchLightning import FluenceReconDataset, Resample

cubes_dataloader = torch.load('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_new_data/version_1/val_dloader.pt')
cubes_dataset,cubes_val_inds = cubes_dataloader.dataset.dataset,cubes_dataloader.dataset.indices

check = NNPL.CheckTrainData(model_cubes,cubes_dataset,np.load(energy_bins))

check.ViewTrainData()

![550keV model cubes pred](Plots/model_cubes_550keV_pred.png)

The model has generally performed well at predicting the bins for this validation data point with some count in an incorrect bin, but of note is that it is still predicting negative values in several bins. In order to avoid this, it may prove useful to add a term to the loss function that penalises any negative bins in the model prediction. 

A more thorough check of the validation data set for both this model and the following will be performed later.

### Cubes and profiles model, 200 samples per dataset, learning rate 0.001

In [9]:
NNPL.PlotLosses([model_cubes_profiles_tloss,model_cubes_profiles_vloss],['Training loss','Validation loss'],'Cubes and profiles model, lr = 0.001, 200 samples per dataset')

![losses](Plots/model_cubes_profiles_200samples.png)

Whilst both the training loss and validation loss do both decrease, it is worth noting that the validation loss here is lower than the training loss, which in turn implies that the model needs to train for longer in this configuration. This is intuitive, as the increased number of bins due to the profile counts will mean there are a greater number of weights for the model to optimize. Other improvements may be found from increasing the learning rate or adding some learning rate scheduling, but otherwise training for longer is realistically required.

It is also worthy of note that the loss is higher than for the cubes-only model, but this may be a factor of the incomplete training. 

In [10]:
fig,ax = plt.subplots(1,3,figsize=(30,10))

l4 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+AmBe_counts,testing_data_dir+AmBe_target,'AmBe','cubes and \n profiles model',ax[0],24,np.load(energy_bins),1,loss_fn)
l5 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+AmLi_counts,testing_data_dir+AmLi_target,'AmLi','cubes and \n profiles model',ax[1],24,np.load(energy_bins),1,loss_fn)
l6 = NNPL.compare_pred_true(model_cubes_profiles,testing_data_dir+'withProfiles/'+Cf_counts,testing_data_dir+Cf_target,r'$^{252}$Cf','cubes and \n profiles model',ax[2],24,np.load(energy_bins),1,loss_fn)

![cubes profiles 200 samples prediction](Plots/model_cubes_profiles_200samples_prediction.png)

In [11]:
print("AmBe loss: {}".format(l4))
print("AmLi loss: {}".format(l5))
print("Cf-252 loss: {}".format(l6))

AmBe loss: 0.09213663637638092
AmLi loss: 0.05291319638490677
Cf-252 loss: 0.04942954331636429


This model also performs poorly on the unseen data, although it has a lower loss than the cubes exclusive model. Most notably, this model predicts a large negative value in the first bin for all three sources, which further motivates introducing a penalty term to the loss to discourage any negative values in the model output. 


Validation data set checking needs some more work before conclusions can be drawn.

In [12]:
cubes_profiles_dataloader = torch.load('/home/nr1315/Documents/Project/MachineLearning/lightning_logs/model_cubes_profiles_new_data/version_5/val_dloader.pt')
cubes_profiles_dataset,cubes_profiles_val_inds = cubes_profiles_dataloader.dataset.dataset,cubes_profiles_dataloader.dataset.indices

check = NNPL.CheckTrainData(model_cubes_profiles,cubes_profiles_dataset,np.load(energy_bins))

check.ViewTrainData()